In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [2]:
df = pd.read_csv('../.csv/digital_marketing_campaign_dataset.csv')
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           8000 non-null   int64  
 1   Age                  8000 non-null   int64  
 2   Gender               8000 non-null   object 
 3   Income               8000 non-null   int64  
 4   CampaignChannel      8000 non-null   object 
 5   CampaignType         8000 non-null   object 
 6   AdSpend              8000 non-null   float64
 7   ClickThroughRate     8000 non-null   float64
 8   ConversionRate       8000 non-null   float64
 9   WebsiteVisits        8000 non-null   int64  
 10  PagesPerVisit        8000 non-null   float64
 11  TimeOnSite           8000 non-null   float64
 12  SocialShares         8000 non-null   int64  
 13  EmailOpens           8000 non-null   int64  
 14  EmailClicks          8000 non-null   int64  
 15  PreviousPurchases    8000 non-null   i

In [3]:
df.head()

,CustomerID,Age,Gender,Income,CampaignChannel,CampaignType,AdSpend,ClickThroughRate,ConversionRate,WebsiteVisits,PagesPerVisit,TimeOnSite,SocialShares,EmailOpens,EmailClicks,PreviousPurchases,LoyaltyPoints,AdvertisingPlatform,AdvertisingTool,Conversion
0,8000,56,Female,136912,Social Media,Awareness,6497.870068,0.043919,0.088031,0,2.399017,7.396803,19,6,9,4,688,IsConfid,ToolConfid,1
1,8001,69,Male,41760,Email,Retention,3898.668606,0.155725,0.182725,42,2.917138,5.352549,5,2,7,2,3459,IsConfid,ToolConfid,1
2,8002,46,Female,88456,PPC,Awareness,1546.429596,0.277490,0.076423,2,8.223619,13.794901,0,11,2,8,2337,IsConfid,ToolConfid,1
3,8003,32,Female,44085,PPC,Conversion,539.525936,0.137611,0.088004,47,4.540939,14.688363,89,2,2,0,2463,IsConfid,ToolConfid,1
4,8004,60,Female,83964,PPC,Conversion,1678.043573,0.252851,0.109940,0,2.046847,13.993370,6,6,6,8,4345,IsConfid,ToolConfid,1


In [4]:
def listar_unicos_simples(df):
    obj_cols = df.select_dtypes(include=['object']).columns

    for col in obj_cols:
        print(f"Coluna: {col}")
        print(f"Valores únicos: {df[col].unique()}")
        print("-" * 30)

# Exemplo de uso (substitua 'df' pelo seu DataFrame):
listar_unicos_simples(df)

Coluna: Gender
Valores únicos: ['Female' 'Male']
------------------------------
Coluna: CampaignChannel
Valores únicos: ['Social Media' 'Email' 'PPC' 'Referral' 'SEO']
------------------------------
Coluna: CampaignType
Valores únicos: ['Awareness' 'Retention' 'Conversion' 'Consideration']
------------------------------
Coluna: AdvertisingPlatform
Valores únicos: ['IsConfid']
------------------------------
Coluna: AdvertisingTool
Valores únicos: ['ToolConfid']
------------------------------


In [5]:

df = df.copy()

assert 'Conversion' in df.columns, "Precisa da coluna Conversion (0/1)."

# 1) KPI base
n_total = len(df)
conv_base = df['Conversion'].mean()
print(f"[BASE] Amostras: {n_total} | Taxa de conversão geral: {conv_base*100:.2f}%\n")

# 2) Por Canal (CampaignChannel)
tmp = (df.groupby('CampaignChannel')
         .agg(total=('Conversion','size'),
              conv_rate=('Conversion','mean'),
              ctr=('ClickThroughRate','mean'))
         .reset_index())
tmp['lift_vs_base'] = tmp['conv_rate'] / conv_base - 1
tmp = tmp.sort_values(['conv_rate','total'], ascending=[False, False])

saida_canais = ["TOP canais por taxa de conversão (com volume e lift vs. base):"]
for _, r in tmp.iterrows():
    saida_canais.append(f"- {r['CampaignChannel']}: conv={r['conv_rate']*100:.2f}% | lift={r['lift_vs_base']*100:+.1f}% | n={int(r['total'])} | CTR médio={r['ctr']:.3f}")
print("\n".join(saida_canais) + "\n")

# 3) Por Tipo (CampaignType)
tp = (df.groupby('CampaignType')
        .agg(total=('Conversion','size'),
             conv_rate=('Conversion','mean'))
        .reset_index())
tp['lift_vs_base'] = tp['conv_rate'] / conv_base - 1
tp = tp.sort_values(['conv_rate','total'], ascending=[False, False])

saida_tipos = ["TOP tipos de campanha por taxa de conversão:"]
for _, r in tp.iterrows():
    saida_tipos.append(f"- {r['CampaignType']}: conv={r['conv_rate']*100:.2f}% | lift={r['lift_vs_base']*100:+.1f}% | n={int(r['total'])}")
print("\n".join(saida_tipos) + "\n")

# 4) Combinações Canal x Tipo (para decidir "onde apostar")
cx = (df.groupby(['CampaignChannel','CampaignType'])
        .agg(total=('Conversion','size'),
             conv_rate=('Conversion','mean'))
        .reset_index())
cx['lift_vs_base'] = cx['conv_rate'] / conv_base - 1

top_combo_conv = cx[cx['total']>=50].sort_values(['conv_rate','total'], ascending=[False, False]).head(10)
top_combo_vol = cx.sort_values('total', ascending=False).head(10)

saida_combo_conv = ["TOP 10 combinações Canal x Tipo por taxa de conversão (min n=50):"]
for _, r in top_combo_conv.iterrows():
    saida_combo_conv.append(f"- {r['CampaignChannel']} x {r['CampaignType']}: conv={r['conv_rate']*100:.2f}% | lift={r['lift_vs_base']*100:+.1f}% | n={int(r['total'])}")
print("\n".join(saida_combo_conv) + "\n")

saida_combo_vol = ["TOP 10 combinações Canal x Tipo por volume:"]
for _, r in top_combo_vol.iterrows():
    saida_combo_vol.append(f"- {r['CampaignChannel']} x {r['CampaignType']}: n={int(r['total'])} | conv={r['conv_rate']*100:.2f}% | lift={r['lift_vs_base']*100:+.1f}%")
print("\n".join(saida_combo_vol) + "\n")

# 5) Gênero
g = (df.groupby('Gender')
       .agg(total=('Conversion','size'),
            conv_rate=('Conversion','mean'))
       .reset_index())
g['lift_vs_base'] = g['conv_rate']/conv_base - 1
g = g.sort_values('conv_rate', ascending=False)

saida_genero = ["Conversão por gênero:"]
for _, r in g.iterrows():
    saida_genero.append(f"- {r['Gender']}: conv={r['conv_rate']*100:.2f}% | lift={r['lift_vs_base']*100:+.1f}% | n={int(r['total'])}")
print("\n".join(saida_genero) + "\n")

# 6) Faixas etárias simples (para falar a linguagem do negócio)
bins_age = [0, 24, 34, 44, 54, 64, np.inf]
labels_age = ['<=24','25-34','35-44','45-54','55-64','65+']
df['AgeBand'] = pd.cut(df['Age'], bins=bins_age, labels=labels_age, right=True, include_lowest=True)

ageg = (df.groupby('AgeBand', observed=True)
          .agg(total=('Conversion','size'),
               conv_rate=('Conversion','mean'))
          .reset_index())
ageg['lift_vs_base'] = ageg['conv_rate']/conv_base - 1
ageg = ageg.sort_values(['conv_rate','total'], ascending=[False, False])

saida_idade = ["Conversão por faixa etária:"]
for _, r in ageg.iterrows():
    saida_idade.append(f"- {r['AgeBand']}: conv={r['conv_rate']*100:.2f}% | lift={r['lift_vs_base']*100:+.1f}% | n={int(r['total'])}")
print("\n".join(saida_idade) + "\n")

# 7) Sinais de intenção (engajamento) — regras fáceis de explicar
df['OpenedEmail'] = (df['EmailOpens'] > 0).astype(int)
df['ClickedEmail'] = (df['EmailClicks'] > 0).astype(int)
df['SharedSocial'] = (df['SocialShares'] > 0).astype(int)

saida_intencao = []
for flag in ['OpenedEmail','ClickedEmail','SharedSocial']:
    t = (df.groupby(flag)['Conversion'].mean()*100).round(2)
    n = df[flag].value_counts().to_dict()
    saida_intencao.append(f"{flag}: conv se 0={t.get(0, np.nan)}% (n={n.get(0,0)}), se 1={t.get(1, np.nan)}% (n={n.get(1,0)})")
print("\n".join(saida_intencao) + "\n")

# 8) Intensidade de navegação (sinal de intenção no site)
def bucketize(series, edges, labels):
    return pd.cut(series, bins=edges, labels=labels, right=True, include_lowest=True)

df['VisitsBand'] = bucketize(df['WebsiteVisits'], [ -1, 0, 2, 5, np.inf], ['0','1-2','3-5','6+'])
df['PagesBand']  = bucketize(df['PagesPerVisit'], [ -np.inf, 1, 2, 3, 5, np.inf], ['<=1','1-2','2-3','3-5','5+'])
df['TimeBand']   = bucketize(df['TimeOnSite'], [ -np.inf, 0.5, 1, 2, 5, np.inf], ['<=0.5','0.5-1','1-2','2-5','5+'])

for col in ['VisitsBand','PagesBand','TimeBand']:
    t = (df.groupby(col, observed=True)['Conversion']
           .agg(conv_rate='mean', total='size')
           .reset_index()
           .sort_values(['conv_rate','total'], ascending=[False, False]))
    
    saida_navegacao = [f"Conversão por {col}:"]
    for _, r in t.iterrows():
        saida_navegacao.append(f"- {r[col]}: conv={r['conv_rate']*100:.2f}% | n={int(r['total'])}")
    print("\n".join(saida_navegacao) + "\n")

# 9) Checklist de onde focar
saida_sugestoes = ["SUGESTÕES DE FOCO (baseadas nos números acima):"]

# 9.1 Canais acima da base com volume
top_channels_focus = tmp[(tmp['lift_vs_base']>0) & (tmp['total']>n_total*0.05)].head(3)
if len(top_channels_focus):
    canais = ", ".join([f"{r['CampaignChannel']} (+{r['lift_vs_base']*100:.0f}% vs base)" for _, r in top_channels_focus.iterrows()])
    saida_sugestoes.append(f"- Canais com boa performance e volume: {canais}")
else:
    saida_sugestoes.append("- Não há canais com lift positivo e volume relevante (>5% do total).")

# 9.2 Tipos de campanha que tracionam
top_types_focus = tp[(tp['lift_vs_base']>0)]
if len(top_types_focus):
    tipos = ", ".join([f"{r['CampaignType']} (+{r['lift_vs_base']*100:.0f}% vs base)" for _, r in top_types_focus.iterrows()])
    saida_sugestoes.append(f"- Tipos que mais tracionam conversão: {tipos}")
else:
    saida_sugestoes.append("- Nenhum tipo com lift acima da base.")

# 9.3 Combinações Canal x Tipo com conv alta e n>=50
if len(top_combo_conv):
    combos = "; ".join([f"{r['CampaignChannel']} x {r['CampaignType']} ({r['conv_rate']*100:.1f}%, n={int(r['total'])})" for _, r in top_combo_conv.iterrows()])
    saida_sugestoes.append(f"- Combinações a priorizar: {combos}")
else:
    saida_sugestoes.append("- Sem combinações com amostra >=50 e conv acima da base.")

# 9.4 Sinais de intenção: clique em e-mail costuma ser forte
click_conv = df.groupby('ClickedEmail')['Conversion'].mean()
if 0 in click_conv and 1 in click_conv:
    lift_click = (click_conv[1]/max(click_conv[0],1e-9)-1)*100
    saida_sugestoes.append(f"- Usuários que clicam em e-mail convertem {lift_click:.0f}% mais que quem não clica.")

print("\n".join(saida_sugestoes))

print("\nFim. Os números acima já respondem: onde investir (canais/tipos/combos), quem engaja mais (gênero/idade), e quais sinais de intenção correlacionam com conversão.")

[BASE] Amostras: 8000 | Taxa de conversão geral: 87.65%

TOP canais por taxa de conversão (com volume e lift vs. base):
- Referral: conv=88.31% | lift=+0.7% | n=1719 | CTR médio=0.152
- PPC: conv=88.28% | lift=+0.7% | n=1655 | CTR médio=0.158
- SEO: conv=87.68% | lift=+0.0% | n=1550 | CTR médio=0.153
- Email: conv=87.03% | lift=-0.7% | n=1557 | CTR médio=0.156
- Social Media: conv=86.83% | lift=-0.9% | n=1519 | CTR médio=0.156

TOP tipos de campanha por taxa de conversão:
- Conversion: conv=93.36% | lift=+6.5% | n=2077
- Retention: conv=85.82% | lift=-2.1% | n=1947
- Awareness: conv=85.56% | lift=-2.4% | n=1988
- Consideration: conv=85.56% | lift=-2.4% | n=1988

TOP 10 combinações Canal x Tipo por taxa de conversão (min n=50):
- SEO x Conversion: conv=94.03% | lift=+7.3% | n=402
- PPC x Conversion: conv=93.96% | lift=+7.2% | n=447
- Email x Conversion: conv=93.27% | lift=+6.4% | n=416
- Referral x Conversion: conv=93.03% | lift=+6.1% | n=445
- Social Media x Conversion: conv=92.37% | l

In [ ]:


features = [
    'Age','Gender','CampaignChannel','CampaignType',
    'ClickThroughRate','WebsiteVisits','PagesPerVisit',
    'TimeOnSite','EmailOpens','EmailClicks','PreviousPurchases'
]

# 2) Criar conjuntos de treino e teste
X = df[features].copy()
y = df['Conversion'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3) Separar colunas categóricas e numéricas
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns
num_cols = [c for c in X_train.columns if c not in cat_cols]

# 4) Aplicar OneHotEncoder apenas nas categóricas
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat = enc.fit_transform(X_train[cat_cols])
X_test_cat = enc.transform(X_test[cat_cols])

# 5) Concatenar categóricas encodadas com numéricas
X_train_enc = np.c_[X_train_cat, X_train[num_cols].to_numpy()]
X_test_enc = np.c_[X_test_cat, X_test[num_cols].to_numpy()]

# 6) Treinar árvore com GridSearch
parameters = {'max_depth': [3, 6, 9, 12, 15, 18, 21]}
decision_tree = DecisionTreeClassifier(criterion='gini', random_state=42, min_samples_leaf=50)
grid_search = GridSearchCV(decision_tree, parameters, n_jobs=-1, cv=3, scoring='roc_auc')
grid_search.fit(X_train_enc, y_train)

print(f"Melhores parâmetros: {grid_search.best_params_}")

# 7) Avaliar no conjunto de teste
best = grid_search.best_estimator_
y_pred = best.predict(X_test_enc)
y_proba = best.predict_proba(X_test_enc)[:,1]

print(f"Acurácia: {accuracy_score(y_test, y_pred):.3f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_proba):.3f}")

Melhores parâmetros: {'max_depth': 6}
Acurácia: 0.874
ROC-AUC: 0.714


In [7]:
import numpy as np
import pandas as pd

# 1) Scores no teste e base
proba = best.predict_proba(X_test_enc)[:,1]
base = y_test.mean()

# 2) Avaliar top k%
ks = [0.10, 0.20, 0.30]  # 10%, 20%, 30%
print(f"Base (teste): conv={base*100:.2f}% | n={len(y_test)}")

for k in ks:
    thr = np.quantile(proba, 1-k)          # limiar para top k%
    mask = proba >= thr
    conv_top = y_test[mask].mean()
    n_top = mask.sum()
    lift_pp = (conv_top - base) * 100
    print(f"Top {int(k*100)}%: conv={conv_top*100:.2f}% | n={n_top} | lift vs base={lift_pp:+.2f} pp")

Base (teste): conv=87.62% | n=1600
Top 10%: conv=94.27% | n=192 | lift vs base=+6.65 pp
Top 20%: conv=93.56% | n=606 | lift vs base=+5.94 pp
Top 30%: conv=93.56% | n=606 | lift vs base=+5.94 pp


In [ ]:

# 1) A qual folha cada amostra do teste pertence
leaf_ids = best.apply(X_test_enc)
df = pd.DataFrame({'leaf': leaf_ids, 'y': y_test.values})

# 2) Taxa de conversão e tamanho por folha
stats = df.groupby('leaf').agg(n=('y','size'), conv=('y','mean')).reset_index()

# 3) Selecionar top 3 folhas com n mínimo
min_n = 100  # ajuste se quiser
top3 = stats[stats.n >= min_n].sort_values(['conv','n'], ascending=[False, False]).head(3)

base = y_test.mean()
print("\nTop folhas por conversão (com n mínimo):")
for _, r in top3.iterrows():
    lift_pp = (r['conv'] - base)*100
    print(f"leaf={int(r['leaf'])} | conv={r['conv']*100:.2f}% | n={int(r['n'])} | lift={lift_pp:+.2f} pp")

# 4) (Opcional) Regra da folha campeã em 1 linha
leaf_star = int(top3.iloc[0]['leaf'])
idx = df.index[df.leaf==leaf_star][0]

# nomes das colunas após o one-hot
feat_names = np.r_[enc.get_feature_names_out(cat_cols), np.array(num_cols)]

path_nodes = best.decision_path(X_test_enc[idx:idx+1]).indices
tree_ = best.tree_
conds = []
for k, node in enumerate(path_nodes):
    if tree_.feature[node] < 0:
        continue
    feat = feat_names[tree_.feature[node]]
    thr  = tree_.threshold[node]
    left, right = tree_.children_left[node], tree_.children_right[node]
    next_node = path_nodes[k+1]
    went_left = (next_node == left)

    # one-hot amigável: "col_cat > 0.5" vira "col = cat"
    if '_' in feat and feat.split('_')[0] in cat_cols:
        base_name = feat.split('_')[0]
        catv = feat[len(base_name)+1:]
        conds.append(f"{base_name} = {catv}" if not went_left else f"{base_name} ≠ {catv}")
    else:
        conds.append(f"{feat} {'<=' if went_left else '>'} {thr:.3f}")

regra = " AND ".join(conds)
conv_star = float(top3.iloc[0]['conv'])
n_star    = int(top3.iloc[0]['n'])
lift_pp   = (conv_star - base)*100

print(f"\nGrupo campeão: {regra} | conv={conv_star*100:.2f}% | n={n_star} | lift vs base={lift_pp:+.2f} pp")


Top folhas por conversão (com n mínimo):
leaf=77 | conv=95.07% | n=142 | lift=+7.45 pp
leaf=79 | conv=93.00% | n=300 | lift=+5.38 pp
leaf=64 | conv=91.30% | n=115 | lift=+3.68 pp

Grupo campeão: PreviousPurchases > 1.500 AND PagesPerVisit > 2.992 AND TimeOnSite > 5.092 AND ClickThroughRate > 0.095 AND Age <= 34.500 AND EmailClicks > 0.500 | conv=95.07% | n=142 | lift vs base=+7.45 pp


In [15]:
import numpy as np
import pandas as pd
import math

# 0) Preparos
feat_names = np.r_[enc.get_feature_names_out(cat_cols), np.array(num_cols)]
tree_ = best.tree_
base = y_test.mean()

# 1) Top 3 folhas por conversão com n mínimo
leaf_ids = best.apply(X_test_enc)
df = pd.DataFrame({'leaf': leaf_ids, 'y': y_test.values})
stats = df.groupby('leaf').agg(n=('y','size'), conv=('y','mean')).reset_index()

min_n = 100  # ajuste se quiser
top3 = stats[stats.n >= min_n].sort_values(['conv','n'], ascending=[False, False]).head(3)

# 2) Helpers de arredondamento e limpeza
def arredonda_regra(nome, op, thr):
    # Mapas para nomes mais amigáveis
    alias = {
        'ClickThroughRate': 'CTR',
        'TimeOnSite': 'Tempo no site (min)',
        'PagesPerVisit': 'Páginas/visita',
        'PreviousPurchases': 'Compras anteriores',
        'EmailClicks': 'Cliques em e-mails',
        'WebsiteVisits': 'Visitas ao site',
        'Age': 'Idade',
    }
    show = alias.get(nome, nome)

    def ge(x):  # imprime ≥
        return f"≥ {x}"
    def le(x):  # imprime ≤
        return f"≤ {x}"

    # Regras por tipo de variável
    if nome in ['PreviousPurchases','EmailClicks','WebsiteVisits','Age','PagesPerVisit']:
        # tratar como contagem/inteiro (PagesPerVisit opera como inteiro prático)
        if op == '>':
            val = math.ceil(thr - 1e-9)  # > 2.99 => 3
            return f"{show} {ge(val)}"
        else:  # '<='
            val = math.floor(thr + 1e-9)  # <= 34.5 => ≤ 34
            return f"{show} {le(val)}"

    if nome == 'ClickThroughRate':
        pct = thr * 100.0
        if op == '>':
            return f"{show} {ge(f'{pct:.1f}%')}"
        else:
            return f"{show} {le(f'{pct:.1f}%')}"

    if nome == 'TimeOnSite':
        # Mostrar em minutos inteiros “do jeito que se fala”
        mins = int(thr)  # > 5.09 => “> 5 min”
        if op == '>':
            return f"{show} > {mins} min"
        else:
            return f"{show} ≤ {mins} min"

    # fallback genérico
    if op == '>':
        return f"{show} > {thr:.3f}"
    else:
        return f"{show} ≤ {thr:.3f}"

def regra_folha_limpa(leaf_id):
    # pegar um índice do teste que caiu na folha
    idx = np.where(leaf_ids == leaf_id)[0][0]
    path_nodes = best.decision_path(X_test_enc[idx:idx+1]).indices

    conds = []
    for k, node in enumerate(path_nodes):
        if tree_.feature[node] < 0:
            continue
        feat = feat_names[tree_.feature[node]]
        thr  = tree_.threshold[node]
        left, right = tree_.children_left[node], tree_.children_right[node]
        next_node = path_nodes[k+1]
        went_left = (next_node == left)
        op = '<=' if went_left else '>'

        # One-hot amigável: "col_cat > 0.5" => "col = cat"; "<= 0.5" => "col ≠ cat"
        base_name = None
        if '_' in feat:
            base_name = feat.split('_')[0]
        if base_name in cat_cols:
            catv = feat[len(base_name)+1:]
            conds.append(f"{base_name} = {catv}" if op == '>' else f"{base_name} ≠ {catv}")
        else:
            conds.append(arredonda_regra(feat, op, thr))
    return " AND ".join(conds)

# 3) Imprimir as 3 regras arredondadas
print("Top 3 grupos:")
for rank, (_, r) in enumerate(top3.iterrows(), start=1):
    leaf_id = int(r['leaf'])
    regra = regra_folha_limpa(leaf_id)
    conv = float(r['conv'])
    n = int(r['n'])
    lift_pp = (conv - base)*100
    print(f"{rank}º) {regra} | conv={conv*100:.2f}% | n={n} | lift vs base={lift_pp:+.2f} pp")

Top 3 grupos:
1º) Compras anteriores ≥ 2 AND Páginas/visita ≥ 3 AND Tempo no site (min) > 5 min AND CTR ≥ 9.5% AND Idade ≤ 34 AND Cliques em e-mails ≥ 1 | conv=95.07% | n=142 | lift vs base=+7.45 pp
2º) Compras anteriores ≥ 2 AND Páginas/visita ≥ 3 AND Tempo no site (min) > 5 min AND CTR ≥ 9.5% AND Idade ≥ 35 AND Compras anteriores ≤ 8 | conv=93.00% | n=300 | lift vs base=+5.38 pp
3º) Compras anteriores ≥ 2 AND Páginas/visita ≥ 3 AND Tempo no site (min) ≤ 5 min AND CTR ≥ 8.0% AND Cliques em e-mails ≥ 3 AND CampaignType ≠ Conversion | conv=91.30% | n=115 | lift vs base=+3.68 pp
